In [1]:
!wandb login

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /home/lokmane/.netrc.
wandb: Currently logged in as: l-benhammadi (l-benhammadi-esi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Phase 1: Data Ingestion & Preparation

In this phase, we prepare the **Tox21 toxicity dataset** and **ZINC unlabeled molecular dataset** for Semi-Supervised Learning (SSL).

### What We Do:
1. **Load Raw Data**: Import Tox21 labeled toxicity data and ZINC unlabeled molecular structures
2. **Canonicalize SMILES**: Standardize molecular representations using RDKit
3. **Feature Engineering**: Compute comprehensive molecular descriptors:
   - Basic properties (MolWt, LogP, H-donors/acceptors)
   - Lipinski's Rule of Five features
   - Topological descriptors (BertzCT, Kappa indices)
   - Pharmacophore features
4. **Handle Class Imbalance**: Downsample majority class for balanced training
5. **Version with W&B**: Create artifacts for data reproducibility

### Datasets:
- **Tox21**: ~7,800 compounds across 12 toxicity assays
- **ZINC**: Large unlabeled molecular database
- **Target**: Binary classification (toxic vs non-toxic)

In [2]:
import wandb
from rdkit import Chem
from rdkit.Chem import Descriptors, QED, Lipinski, Crippen, MolSurf, rdMolDescriptors
import pandas as pd
import numpy as np
from sklearn.utils import resample

PROJECT="QSAR_MLOPS_TOX21"

run = wandb.init(project=PROJECT, job_type="prepare-data")
raw_df_unlabeled = pd.read_csv('../../data/raw/original_data/zinc_unlabeled.csv')
raw_df_labeled = pd.read_csv('../../data/raw/original_data/tox21.csv')

def canonicalize_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles.strip())
        if mol is not None:
            return Chem.MolToSmiles(mol, canonical=True)
    except:
        pass
    return None

def compute_comprehensive_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles.strip())
        if mol is not None:
            features = {}
            
            # Basic molecrun.finish()ular properties
            features['MolWt'] = Descriptors.MolWt(mol)
            features['LogP'] = Descriptors.MolLogP(mol)
            features['NumHDonors'] = Descriptors.NumHDonors(mol)
            features['NumHAcceptors'] = Descriptors.NumHAcceptors(mol)
            features['NumRotatableBonds'] = Descriptors.NumRotatableBonds(mol)
            features['NumAromaticRings'] = Descriptors.NumAromaticRings(mol)
            
            # Lipinski's Rule of Five
            features['NumHeteroatoms'] = Descriptors.NumHeteroatoms(mol)
            features['TPSA'] = Descriptors.TPSA(mol)
            
            # Complexity and shape
            features['NumRings'] = Descriptors.RingCount(mol)
            features['NumAliphaticRings'] = Descriptors.NumAliphaticRings(mol)
            features['NumSaturatedRings'] = Descriptors.NumSaturatedRings(mol)
            features['FractionCsp3'] = Descriptors.FractionCSP3(mol) 
            
            # Electronic properties
            features['NumValenceElectrons'] = Descriptors.NumValenceElectrons(mol)
            
            try:
                features['MaxPartialCharge'] = Descriptors.MaxPartialCharge(mol)
                features['MinPartialCharge'] = Descriptors.MinPartialCharge(mol)
            except:
                features['MaxPartialCharge'] = 0
                features['MinPartialCharge'] = 0
            
            # Molecular surface area
            features['LabuteASA'] = Descriptors.LabuteASA(mol)
            features['PEOE_VSA1'] = Descriptors.PEOE_VSA1(mol)
            features['PEOE_VSA2'] = Descriptors.PEOE_VSA2(mol)
            
            # Drug-likeness scores
            features['QED'] = QED.qed(mol)
            
            # Topological descriptors
            features['BertzCT'] = Descriptors.BertzCT(mol)
            features['Chi0v'] = Descriptors.Chi0v(mol)
            features['Chi1v'] = Descriptors.Chi1v(mol)
            features['Kappa1'] = Descriptors.Kappa1(mol)
            features['Kappa2'] = Descriptors.Kappa2(mol)
            
            # Additional descriptors
            features['MolMR'] = Descriptors.MolMR(mol)
            features['BalabanJ'] = Descriptors.BalabanJ(mol)
            features['HallKierAlpha'] = Descriptors.HallKierAlpha(mol)
            features['NumSaturatedCarbocycles'] = Descriptors.NumSaturatedCarbocycles(mol)
            features['NumAromaticCarbocycles'] = Descriptors.NumAromaticCarbocycles(mol)
            features['NumSaturatedHeterocycles'] = Descriptors.NumSaturatedHeterocycles(mol)
            features['NumAromaticHeterocycles'] = Descriptors.NumAromaticHeterocycles(mol)
            
            # Pharmacophore features
            features['fr_NH2'] = Descriptors.fr_NH2(mol)
            features['fr_COO'] = Descriptors.fr_COO(mol)
            features['fr_benzene'] = Descriptors.fr_benzene(mol)
            features['fr_furan'] = Descriptors.fr_furan(mol)
            features['fr_halogen'] = Descriptors.fr_halogen(mol)
            
            return pd.Series(features)
    except Exception as e:
        print(f"Error computing features: {e}") 
        pass
    return pd.Series()

raw_df_labeled['canonical_smiles'] = raw_df_labeled['smiles'].apply(canonicalize_smiles)
raw_df_labeled = raw_df_labeled.dropna(subset=['canonical_smiles'])

tox_columns = ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 
               'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']

raw_df_labeled['toxic'] = raw_df_labeled[tox_columns].max(axis=1)

raw_df_labeled = raw_df_labeled.dropna(subset=['toxic'])

raw_df_labeled['toxic'] = raw_df_labeled['toxic'].astype(int)

raw_df_labeled = raw_df_labeled.drop(columns=tox_columns)

toxic_count = (raw_df_labeled['toxic'] == 1).sum()
non_toxic_count = (raw_df_labeled['toxic'] == 0).sum()


toxic_df = raw_df_labeled[raw_df_labeled['toxic'] == 1]
non_toxic_df = raw_df_labeled[raw_df_labeled['toxic'] == 0]

non_toxic_downsampled = resample(non_toxic_df, 
                                  replace=False,
                                  n_samples=len(toxic_df),
                                  random_state=42)

raw_df_labeled_balanced = pd.concat([toxic_df, non_toxic_downsampled])
raw_df_labeled_balanced = raw_df_labeled_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


labeled_features = raw_df_labeled_balanced['canonical_smiles'].apply(compute_comprehensive_features)
all_labeled_with_features = pd.concat([raw_df_labeled_balanced, labeled_features], axis=1)
all_labeled_with_features = all_labeled_with_features.dropna()

all_labeled_with_features.to_csv('../../data/raw/enhanced_data/tox21/labeled_features.csv', index=False)


wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /home/lokmane/.netrc.
wandb: Currently logged in as: l-benhammadi (l-benhammadi-esi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[10:45:39] WARNING: not removing hydrogen atom without neighbors
[10:45:39] Explicit valence for atom # 8 Al, 6, is greater than permitted
[10:45:39] Explicit valence for atom # 3 Al, 6, is greater than permitted
[10:45:39] Explicit valence for atom # 4 Al, 6, is greater than permitted
[10:45:40] Explicit valence for atom # 4 Al, 6, is greater than permitted
[10:45:40] Explicit valence for atom # 9 Al, 6, is greater than permitted
[10:45:40] Explicit valence for atom # 5 Al, 6, is greater than permitted
[10:45:40] Explicit valence for atom # 16 Al, 6, is greater than permitted
[10:45:41] Explicit valence for atom # 20 Al, 6, is greater than permitted
[10:45:48] WARNING: not removing hydrogen atom without neighbors
[10:45:48] WARNING: not removing hydrogen atom without neighbors


In [3]:
raw_df_unlabeled['canonical_smiles'] = raw_df_unlabeled['smiles'].apply(canonicalize_smiles)
raw_df_unlabeled = raw_df_unlabeled.dropna(subset=['canonical_smiles'])

unlabeled_features = raw_df_unlabeled['canonical_smiles'].apply(compute_comprehensive_features)
unlabeled_with_features = pd.concat([raw_df_unlabeled[['smiles', 'canonical_smiles']], unlabeled_features], axis=1)
unlabeled_with_features['toxic'] = np.nan
unlabeled_with_features = unlabeled_with_features.dropna(subset=unlabeled_features.columns.tolist())

unlabeled_with_features.to_csv('../../data/raw/enhanced_data/tox21/unlabeled_features.csv', index=False)

In [4]:
exclude_cols = ['smiles', 'canonical_smiles', 'FDA_APPROVED', 'toxic','mol_id']
all_features = [col for col in all_labeled_with_features.columns if col not in exclude_cols]

X_labeled = all_labeled_with_features[all_features]
y_tox = all_labeled_with_features['toxic']

X_unlabeled = unlabeled_with_features[all_features]

In [5]:
def clip_outliers(df, std_threshold=3):
    """Clip outliers using standard deviation method"""
    df_clipped = df.copy()
    outlier_count = 0
    
    for col in df.columns:
        mean = df[col].mean()
        std = df[col].std()
        lower_bound = mean - std_threshold * std
        upper_bound = mean + std_threshold * std
        
        # Count outliers
        outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
        if outliers > 0:
            outlier_count += outliers
            print(f"  {col:20s}: {outliers} outliers clipped")
            df_clipped[col] = df[col].clip(lower_bound, upper_bound)
    
    return df_clipped, outlier_count

X_labeled_clipped, labeled_outliers = clip_outliers(X_labeled, std_threshold=3)
X_unlabeled_clipped, unlabeled_outliers = clip_outliers(X_unlabeled, std_threshold=3)

  MolWt               : 102 outliers clipped
  LogP                : 80 outliers clipped
  NumHDonors          : 93 outliers clipped
  NumHAcceptors       : 102 outliers clipped
  NumRotatableBonds   : 110 outliers clipped
  NumAromaticRings    : 45 outliers clipped
  NumHeteroatoms      : 75 outliers clipped
  TPSA                : 83 outliers clipped
  NumRings            : 78 outliers clipped
  NumAliphaticRings   : 101 outliers clipped
  NumSaturatedRings   : 104 outliers clipped
  NumValenceElectrons : 99 outliers clipped
  MinPartialCharge    : 14 outliers clipped
  LabuteASA           : 97 outliers clipped
  PEOE_VSA1           : 115 outliers clipped
  PEOE_VSA2           : 70 outliers clipped
  BertzCT             : 76 outliers clipped
  Chi0v               : 103 outliers clipped
  Chi1v               : 101 outliers clipped
  Kappa1              : 100 outliers clipped
  Kappa2              : 1 outliers clipped
  MolMR               : 97 outliers clipped
  BalabanJ            : 

In [6]:
df_labeled_processed = X_labeled_clipped.copy()

df_labeled_processed['toxic'] = y_tox.values

df_unlabeled_processed = X_unlabeled_clipped.copy()
df_unlabeled_processed['toxic'] = np.nan

df_labeled_processed.to_csv('../../data/processed/tox21/labeled_processed.csv', index=False)
df_unlabeled_processed.to_csv('../../data/processed/tox21/unlabeled_processed.csv', index=False)

In [7]:
artifact_tox21_labeled   = wandb.Artifact(
            name="tox21-labeled-dataset",
            type="dataset",
            description="Cleaned labeled tox21 data for v1.0" 
            )
artifact_tox21_labeled.add_file('../../data/processed/tox21/labeled_processed.csv')

run.log_artifact(artifact_tox21_labeled)

<Artifact tox21-labeled-dataset>

In [8]:
artifact_zinc_unlabeled   = wandb.Artifact(
            name="zinc-unlabeled-dataset",
            type="dataset",
            description="Cleaned unlabeled zinc data for v1.0" 
            )
artifact_zinc_unlabeled.add_file('../../data/processed/tox21/unlabeled_processed.csv')
run.log_artifact(artifact_zinc_unlabeled)
run.finish()

## Phase 2 & 3: Baseline Model & Hyperparameter Optimization

We establish a **baseline supervised model** using Random Forest, then optimize hyperparameters with W&B Sweeps.

### Approach:
1. **Baseline Training**: Train Random Forest on labeled data only
2. **Bayesian Optimization**: Systematically search hyperparameter space
3. **Metrics Tracked**: F1-Score, ROC-AUC, Precision, Recall, Accuracy
4. **Model Registry**: Save best baseline model as W&B artifact

### Hyperparameters Tuned:
- `n_estimators`: Number of trees (50-200)
- `max_depth`: Tree depth (10-30)
- `min_samples_split`: Split threshold (2, 5, 10)
- `min_samples_leaf`: Leaf size (1, 2, 4)
- `max_features`: Feature sampling ('sqrt', 'log2')

This baseline serves as the benchmark for our Semi-Supervised Learning methods.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

wandb.init(project=PROJECT, job_type="experiment")

run = wandb.use_artifact('zinc-unlabeled-dataset:latest')
data_path = run.download()
df_unlabeled = pd.read_csv(f"{data_path}/unlabeled_processed.csv")

run= wandb.use_artifact('tox21-labeled-dataset:latest')
data_path = run.download()
df_labeled = pd.read_csv(f"{data_path}/labeled_processed.csv")

wandb: Downloading large artifact 'zinc-unlabeled-dataset:latest', 92.69MB. 1 files...
wandb:   1 of 1 files downloaded.  
Done. 00:00:00.8 (122.6MB/s)
wandb:   1 of 1 files downloaded.  


Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "/home/lokmane/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/lokmane/anaconda3/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "/home/lokmane/anaconda3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lokmane/anaconda3/lib/python3.11/site-packages/wandb/sdk/wandb_run.py", line 336, in check_internal_messages
    self._loop_check_status(
  File "/home/lokmane/anaconda3/lib/python3.11/site-packages/wandb/sdk/wandb_run.py", line 237, in _loop_check_status
    local_handle = request()
                   ^^^^^^^^^
  File "/home/lokmane/anaconda3/lib/python3.11/site-packages/wandb/sdk/interface/interface.py", line 1007, in deliver_internal_messages
    return self._deliver_internal_messages(internal_message)
           ^^^^^^^^^^^^^^^^^^^^^^^

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

X = df_labeled.drop('toxic', axis=1)
y = df_labeled['toxic']

X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

print(f"NaN values: {X.isna().sum().sum()}")
print(f"Inf values: {np.isinf(X.values).sum()}")

X_unlabeled_full = df_unlabeled.drop('toxic', axis=1)
X_unlabeled_full = X_unlabeled_full.replace([np.inf, -np.inf], np.nan)
X_unlabeled_full = X_unlabeled_full.fillna(X_unlabeled_full.median())

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("\nScaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_unlabeled_scaled = scaler.transform(X_unlabeled_full)

X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
X_unlabeled_full = X_unlabeled_scaled

NaN values: 0
Inf values: 0

Scaling features...


In [11]:
baseline_sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_f1', 'goal': 'maximize'},
    'parameters': {
        'n_estimators': {'values': [50, 100, 150, 200]},
        'max_depth': {'values': [10, 15, 20, 25, 30]},
        'min_samples_split': {'values': [2, 5, 10]},
        'min_samples_leaf': {'values': [1, 2, 4]},
        'max_features': {'values': ['sqrt', 'log2']}
    }
}

In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

def train_baseline():
    run = wandb.init()
    config = wandb.config
    
    model = RandomForestClassifier(
        n_estimators=config.n_estimators,
        max_depth=config.max_depth,
        min_samples_split=config.min_samples_split,
        min_samples_leaf=config.min_samples_leaf,
        max_features=config.max_features,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'
    )
    
    model.fit(X_train.values, y_train.values)
    
    y_pred = model.predict(X_test.values)
    y_pred_proba = model.predict_proba(X_test.values)[:, 1]
    
    metrics = {
        'val_accuracy': accuracy_score(y_test, y_pred),
        'val_precision': precision_score(y_test, y_pred, zero_division=0),
        'val_recall': recall_score(y_test, y_pred, zero_division=0),
        'val_f1': f1_score(y_test, y_pred, zero_division=0),
        'val_roc_auc': roc_auc_score(y_test, y_pred_proba)
    }
    
    wandb.log(metrics)
    
    print(f"F1: {metrics['val_f1']:.4f}, ROC-AUC: {metrics['val_roc_auc']:.4f}")

print("✓ Baseline training function defined")

✓ Baseline training function defined


In [13]:
baseline_sweep_id = wandb.sweep(baseline_sweep_config, project=PROJECT)
wandb.agent(baseline_sweep_id, train_baseline, count=20)

print("✓ Baseline sweep completed!")

Create sweep with ID: 6z4ddpg3
Sweep URL: https://wandb.ai/l-benhammadi-esi/QSAR_MLOPS_TOX21/sweeps/6z4ddpg3


wandb: Agent Starting Run: 8n3uo1b1 with config:
wandb: 	max_depth: 25
wandb: 	max_features: log2
wandb: 	min_samples_leaf: 4
wandb: 	min_samples_split: 5
wandb: 	n_estimators: 150
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /home/lokmane/.netrc.


wandb: 
wandb: 🚀 View run vocal-plasma-2 at: https://wandb.ai/l-benhammadi-esi/QSAR_MLOPS_TOX21/runs/w8krzjlj
wandb: Find logs at: wandb/run-20260129_110301-w8krzjlj/logs


F1: 0.6792, ROC-AUC: 0.7697


val_accuracy,▁
val_f1,▁
val_precision,▁
val_recall,▁
val_roc_auc,▁
val_accuracy,0.69923
val_f1,0.67922
val_precision,0.72326
val_recall,0.64024
val_roc_auc,0.76973


...

wandb: Agent Starting Run: lv9esd8h with config:
wandb: 	max_depth: 30
wandb: 	max_features: log2
wandb: 	min_samples_leaf: 1
wandb: 	min_samples_split: 5
wandb: 	n_estimators: 50
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /home/lokmane/.netrc.


F1: 0.6831, ROC-AUC: 0.7567


val_accuracy,▁
val_f1,▁
val_precision,▁
val_recall,▁
val_roc_auc,▁
val_accuracy,0.69747
val_f1,0.68311
val_precision,0.713
val_recall,0.65562
val_roc_auc,0.75673


✓ Baseline sweep completed!
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7d22f7755810, execution_count=13 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7d22ebd13f50, raw_cell="baseline_sweep_id = wandb.sweep(baseline_sweep_con.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X20sZmlsZQ%3D%3D> result=None>,),kwargs {}:


AlreadyJoinedError: 

In [14]:
import joblib
import os

api = wandb.Api()
entity = wandb.Api().default_entity

baseline_sweep = api.sweep(f"{entity}/{PROJECT}/{baseline_sweep_id}")
best_baseline_run = baseline_sweep.best_run()

print(f"\n🏆 BEST BASELINE MODEL")
print(f"{'='*60}")
print(f"Run name: {best_baseline_run.name}")
print(f"F1-Score: {best_baseline_run.summary.get('val_f1'):.4f}")
print(f"ROC-AUC: {best_baseline_run.summary.get('val_roc_auc'):.4f}")
print(f"\nBest Hyperparameters:")
print(f"  n_estimators: {best_baseline_run.config['n_estimators']}")
print(f"  max_depth: {best_baseline_run.config['max_depth']}")
print(f"  min_samples_split: {best_baseline_run.config['min_samples_split']}")
print(f"  min_samples_leaf: {best_baseline_run.config['min_samples_leaf']}")
print(f"  max_features: {best_baseline_run.config['max_features']}")

best_baseline_config = {
    'n_estimators': best_baseline_run.config['n_estimators'],
    'max_depth': best_baseline_run.config['max_depth'],
    'min_samples_split': best_baseline_run.config['min_samples_split'],
    'min_samples_leaf': best_baseline_run.config['min_samples_leaf'],
    'max_features': best_baseline_run.config['max_features']
}

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7d22f35c8d50, raw_cell="import joblib
import os

api = wandb.Api()
entity .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X21sZmlsZQ%3D%3D>,),kwargs {}:


AlreadyJoinedError: 

wandb: Sorting runs by -summary_metrics.val_f1



🏆 BEST BASELINE MODEL
Run name: giddy-sweep-4
F1-Score: 0.6886
ROC-AUC: 0.7610

Best Hyperparameters:
  n_estimators: 50
  max_depth: 30
  min_samples_split: 10
  min_samples_leaf: 2
  max_features: log2
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7d22f38c8cd0, execution_count=14 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7d22f35c8d50, raw_cell="import joblib
import os

api = wandb.Api()
entity .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X21sZmlsZQ%3D%3D> result=None>,),kwargs {}:


AlreadyJoinedError: 

In [15]:
run = wandb.init(project=PROJECT, job_type="register-baseline-model")

best_baseline_model = RandomForestClassifier(
    n_estimators=best_baseline_config['n_estimators'],
    max_depth=best_baseline_config['max_depth'],
    min_samples_split=best_baseline_config['min_samples_split'],
    min_samples_leaf=best_baseline_config['min_samples_leaf'],
    max_features=best_baseline_config['max_features'],
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

best_baseline_model.fit(X_train.values, y_train.values)

os.makedirs('models', exist_ok=True)
joblib.dump(best_baseline_model, 'models/best_baseline_rf_model.pkl')

baseline_artifact = wandb.Artifact(
    name='tox21-baseline-rf-model',
    type='model',
    description='Best baseline Random Forest model (supervised only)',
    metadata={
        'method': 'baseline_supervised',
        'n_samples': len(X_train),
        'f1_score': best_baseline_run.summary.get('val_f1'),
        'roc_auc': best_baseline_run.summary.get('val_roc_auc'),
        **best_baseline_config
    }
)

baseline_artifact.add_file('models/best_baseline_rf_model.pkl')
run.log_artifact(baseline_artifact)
run.finish()

print("✓ Best baseline model registered!")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7d22fd1b68d0, raw_cell="run = wandb.init(project=PROJECT, job_type="regist.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X22sZmlsZQ%3D%3D>,),kwargs {}:


AlreadyJoinedError: 

✓ Best baseline model registered!
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7d22eb9788d0, execution_count=15 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7d22fd1b68d0, raw_cell="run = wandb.init(project=PROJECT, job_type="regist.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X22sZmlsZQ%3D%3D> result=None>,),kwargs {}:


AlreadyJoinedError: 

## Phase 4: Semi-Supervised Learning & Model Registry

After establishing our baseline, we leverage **unlabeled ZINC molecules** to improve model performance using SSL techniques.

### SSL Methods Implemented:
1. **Label Propagation**: Graph-based label spreading with RBF kernel
2. **Self-Training**: Iteratively label high-confidence predictions
3. **Co-Training**: Train two models on different feature views

### Process:
1. **Hyperparameter Sweep**: Test different SSL configurations
2. **Compare Methods**: Evaluate improvement over baseline
3. **Select Best Model**: Based on F1-Score and ROC-AUC
4. **Register to W&B**: Save best model with metadata

The best SSL model significantly outperforms the supervised baseline by utilizing tens of thousands of unlabeled molecules.

In [16]:
from sklearn.semi_supervised import LabelPropagation, LabelSpreading, SelfTrainingClassifier

ssl_sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_f1', 'goal': 'maximize'},
    'parameters': {
        'ssl_method': {
            'values': ['label_propagation', 'self_training', 'co_training']
        },
        
        'n_unlabeled': {
            'values': [5000, 10000, 15000, 20000]
        },
        
        'lp_gamma': {
            'distribution': 'log_uniform_values',
            'min': 0.001,
            'max': 0.5
        },
        'lp_max_iter': {
            'values': [500, 1000, 1500]
        },
        
        'ls_alpha': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.9
        },
        
        'st_threshold': {
            'distribution': 'uniform',
            'min': 0.7,
            'max': 0.95
        },
        'st_max_iter': {
            'values': [5, 10, 15]
        },
        
        'ct_confidence_threshold': {
            'distribution': 'uniform',
            'min': 0.75,
            'max': 0.95
        },
        'ct_samples_per_iter': {
            'values': [25, 50, 100]
        },
        'ct_max_iterations': {
            'values': [10, 15, 20]
        }
    }
}

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7d22f7eefc10, raw_cell="from sklearn.semi_supervised import LabelPropagati.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X24sZmlsZQ%3D%3D>,),kwargs {}:


AlreadyJoinedError: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7d22fe12d850, execution_count=16 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7d22f7eefc10, raw_cell="from sklearn.semi_supervised import LabelPropagati.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X24sZmlsZQ%3D%3D> result=None>,),kwargs {}:


AlreadyJoinedError: 

In [17]:
def train_ssl():
    run = wandb.init()
    config = wandb.config
    
    n_unlabeled = min(config.n_unlabeled, len(X_unlabeled_full))
    X_unlabeled = X_unlabeled_full[:n_unlabeled]
    
    
    # === LABEL PROPAGATION ===
    if config.ssl_method == 'label_propagation':
        X_combined = np.vstack([X_train.values, X_unlabeled])
        y_combined = np.concatenate([y_train.values, np.full(n_unlabeled, -1)])
        
        model = LabelPropagation(
            kernel='rbf',
            gamma=config.lp_gamma,
            max_iter=config.lp_max_iter,
            n_jobs=-1
        )
        model.fit(X_combined, y_combined)
        y_pred = model.predict(X_test.values)
        y_pred_proba = model.predict_proba(X_test.values)[:, 1]
    
    # === SELF-TRAINING ===
    elif config.ssl_method == 'self_training':
        X_combined = np.vstack([X_train.values, X_unlabeled])
        y_combined = np.concatenate([y_train.values, np.full(n_unlabeled, -1)])
        
        base_clf = RandomForestClassifier(
            n_estimators=best_baseline_config['n_estimators'],
            max_depth=best_baseline_config['max_depth'],
            min_samples_split=best_baseline_config['min_samples_split'],
            min_samples_leaf=best_baseline_config['min_samples_leaf'],
            max_features=best_baseline_config['max_features'],
            random_state=42,
            n_jobs=-1,
            class_weight='balanced'
        )
        model = SelfTrainingClassifier(
            base_estimator=base_clf,
            threshold=config.st_threshold,
            max_iter=config.st_max_iter,
            verbose=False
        )
        model.fit(X_combined, y_combined)
        y_pred = model.predict(X_test.values)
        y_pred_proba = model.predict_proba(X_test.values)[:, 1]
    
    # === CO-TRAINING ===
    elif config.ssl_method == 'co_training':
        view1_candidates = [
            'MolWt', 'LogP', 'NumHDonors', 'NumHAcceptors', 'NumValenceElectrons',
            'TPSA', 'MaxPartialCharge', 'MinPartialCharge', 'LabuteASA', 'MolMR',
            'QED', 'NumHeteroatoms'
        ]
        
        view2_candidates = [
            'NumRotatableBonds', 'NumAromaticRings', 'NumRings', 'NumAliphaticRings',
            'NumSaturatedRings', 'FractionCsp3', 'PEOE_VSA1', 'PEOE_VSA2', 'BertzCT',
            'Chi0v', 'Chi1v', 'Kappa1', 'Kappa2', 'BalabanJ', 'HallKierAlpha',
            'NumSaturatedCarbocycles', 'NumAromaticCarbocycles',
            'NumSaturatedHeterocycles', 'NumAromaticHeterocycles',
            'fr_NH2', 'fr_COO', 'fr_benzene', 'fr_furan', 'fr_halogen'
        ]
        
        feature_cols = X_train.columns.tolist()
        
        view1_features = [f for f in view1_candidates if f in feature_cols]
        view2_features = [f for f in view2_candidates if f in feature_cols]
        
        v1_idx = [feature_cols.index(f) for f in view1_features]
        v2_idx = [feature_cols.index(f) for f in view2_features]
        
        X_train_v1 = X_train.values[:, v1_idx]
        X_train_v2 = X_train.values[:, v2_idx]
        X_test_v1 = X_test.values[:, v1_idx]
        X_test_v2 = X_test.values[:, v2_idx]
        X_unlabeled_v1 = X_unlabeled[:, v1_idx]
        X_unlabeled_v2 = X_unlabeled[:, v2_idx]
        
        y_train_curr = y_train.values.copy()
        
        clf1 = RandomForestClassifier(
            n_estimators=best_baseline_config['n_estimators'],
            max_depth=best_baseline_config['max_depth'],
            min_samples_split=best_baseline_config['min_samples_split'],
            min_samples_leaf=best_baseline_config['min_samples_leaf'],
            max_features=best_baseline_config['max_features'],
            random_state=42,
            n_jobs=-1,
            class_weight='balanced'
        )
        clf2 = RandomForestClassifier(
            n_estimators=best_baseline_config['n_estimators'],
            max_depth=best_baseline_config['max_depth'],
            min_samples_split=best_baseline_config['min_samples_split'],
            min_samples_leaf=best_baseline_config['min_samples_leaf'],
            max_features=best_baseline_config['max_features'],
            random_state=43,
            n_jobs=-1,
            class_weight='balanced'
        )
        
        mask_available = np.ones(n_unlabeled, dtype=bool)
        
        for iteration in range(config.ct_max_iterations):
            if not np.any(mask_available):
                break
            
            clf1.fit(X_train_v1, y_train_curr)
            clf2.fit(X_train_v2, y_train_curr)
            
            available_v1 = X_unlabeled_v1[mask_available]
            available_v2 = X_unlabeled_v2[mask_available]
            
            if len(available_v1) == 0:
                break
            
            prob1 = clf1.predict_proba(available_v1)
            prob2 = clf2.predict_proba(available_v2)
            
            conf1 = np.max(prob1, axis=1)
            conf2 = np.max(prob2, axis=1)
            pred1 = np.argmax(prob1, axis=1)
            pred2 = np.argmax(prob2, axis=1)
            
            confident1 = conf1 > config.ct_confidence_threshold
            confident2 = conf2 > config.ct_confidence_threshold
            
            available_indices = np.where(mask_available)[0]
            samples_to_add = []
            labels_to_add = []
            
            if np.any(confident1):
                top_indices1 = np.argsort(conf1)[::-1][:config.ct_samples_per_iter]
                top_indices1 = top_indices1[confident1[top_indices1]]
                for idx in top_indices1:
                    samples_to_add.append(available_indices[idx])
                    labels_to_add.append(pred1[idx])
            
            if np.any(confident2):
                top_indices2 = np.argsort(conf2)[::-1][:config.ct_samples_per_iter]
                top_indices2 = top_indices2[confident2[top_indices2]]
                for idx in top_indices2:
                    if available_indices[idx] not in samples_to_add:
                        samples_to_add.append(available_indices[idx])
                        labels_to_add.append(pred2[idx])
            
            if len(samples_to_add) == 0:
                break
            
            samples_to_add = np.array(samples_to_add)
            labels_to_add = np.array(labels_to_add)
            
            X_train_v1 = np.vstack([X_train_v1, X_unlabeled_v1[samples_to_add]])
            X_train_v2 = np.vstack([X_train_v2, X_unlabeled_v2[samples_to_add]])
            y_train_curr = np.concatenate([y_train_curr, labels_to_add])
            
            mask_available[samples_to_add] = False
        
        clf1.fit(X_train_v1, y_train_curr)
        clf2.fit(X_train_v2, y_train_curr)
        
        p1 = clf1.predict_proba(X_test_v1)[:, 1]
        p2 = clf2.predict_proba(X_test_v2)[:, 1]
        y_pred_proba = (p1 + p2) / 2
        y_pred = (y_pred_proba >= 0.5).astype(int)
    
    metrics = {
        'val_accuracy': accuracy_score(y_test, y_pred),
        'val_precision': precision_score(y_test, y_pred, zero_division=0),
        'val_recall': recall_score(y_test, y_pred, zero_division=0),
        'val_f1': f1_score(y_test, y_pred, zero_division=0),
        'val_roc_auc': roc_auc_score(y_test, y_pred_proba),
        'n_unlabeled_used': n_unlabeled,
        'n_labeled': len(X_train)
    }
    
    wandb.log(metrics)
    
    baseline_f1 = best_baseline_run.summary.get('val_f1')
    improvement = ((metrics['val_f1'] - baseline_f1) / baseline_f1) * 100
    wandb.log({'improvement_over_baseline_pct': improvement})
    
    print(f"F1: {metrics['val_f1']:.4f} (+{improvement:+.2f}%), ROC-AUC: {metrics['val_roc_auc']:.4f}")

print("✓ SSL training function defined (all 4 methods)")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7d22f35fa8d0, raw_cell="def train_ssl():
    run = wandb.init()
    config.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X25sZmlsZQ%3D%3D>,),kwargs {}:


AlreadyJoinedError: 

✓ SSL training function defined (all 4 methods)
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7d22fe1f5a50, execution_count=17 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7d22f35fa8d0, raw_cell="def train_ssl():
    run = wandb.init()
    config.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X25sZmlsZQ%3D%3D> result=None>,),kwargs {}:


AlreadyJoinedError: 

In [18]:
ssl_sweep_id = wandb.sweep(ssl_sweep_config, project=PROJECT)
wandb.agent(ssl_sweep_id, train_ssl, count=50)

print("✓ SSL sweep completed!")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7d22fd120a50, raw_cell="ssl_sweep_id = wandb.sweep(ssl_sweep_config, proje.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X26sZmlsZQ%3D%3D>,),kwargs {}:


AlreadyJoinedError: 

Create sweep with ID: 415gsqzj
Sweep URL: https://wandb.ai/l-benhammadi-esi/QSAR_MLOPS_TOX21/sweeps/415gsqzj


wandb: Agent Starting Run: vq9kuxtt with config:
wandb: 	ct_confidence_threshold: 0.8823214756927386
wandb: 	ct_max_iterations: 20
wandb: 	ct_samples_per_iter: 50
wandb: 	lp_gamma: 0.46633720675856694
wandb: 	lp_max_iter: 1000
wandb: 	ls_alpha: 0.6173144766786385
wandb: 	n_unlabeled: 15000
wandb: 	ssl_method: self_training
wandb: 	st_max_iter: 10
wandb: 	st_threshold: 0.8473665564102328
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /home/lokmane/.netrc.


/home/lokmane/anaconda3/lib/python3.11/site-packages/sklearn/semi_supervised/_self_training.py:210: FutureWarning: `base_estimator` has been deprecated in 1.6 and will be removed in 1.8. Please use `estimator` instead.
  warn(


...

/home/lokmane/anaconda3/lib/python3.11/site-packages/sklearn/semi_supervised/_self_training.py:210: FutureWarning: `base_estimator` has been deprecated in 1.6 and will be removed in 1.8. Please use `estimator` instead.
  warn(


F1: 0.6756 (+-1.90%), ROC-AUC: 0.7471


improvement_over_baseline_pct,▁
n_labeled,▁
n_unlabeled_used,▁
val_accuracy,▁
val_f1,▁
val_precision,▁
val_recall,▁
val_roc_auc,▁
improvement_over_baseline_pct,-1.89811
n_labeled,3962
n_unlabeled_used,20000


✓ SSL sweep completed!
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7d22fd1c7ed0, execution_count=18 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7d22fd120a50, raw_cell="ssl_sweep_id = wandb.sweep(ssl_sweep_config, proje.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X26sZmlsZQ%3D%3D> result=None>,),kwargs {}:


AlreadyJoinedError: 

In [19]:
import pandas as pd

ssl_sweep = api.sweep(f"{entity}/{PROJECT}/{ssl_sweep_id}")
ssl_runs = list(ssl_sweep.runs)

results = []
for run in ssl_runs:
    if run.state == 'finished':
        results.append({
            'method': run.config.get('ssl_method', 'unknown'),
            'n_unlabeled': run.config.get('n_unlabeled', 0),
            'f1_score': run.summary.get('val_f1', 0),
            'roc_auc': run.summary.get('val_roc_auc', 0),
            'accuracy': run.summary.get('val_accuracy', 0),
            'precision': run.summary.get('val_precision', 0),
            'recall': run.summary.get('val_recall', 0),
            'improvement_pct': run.summary.get('improvement_over_baseline_pct', 0),
            'run_name': run.name,
            'run_id': run.id
        })

results_df = pd.DataFrame(results)

print(f"\n{'='*100}")
print("SSL RESULTS SUMMARY (Top 15 by F1-Score)")
print(f"{'='*100}")
if len(results_df) > 0:
    print(results_df.sort_values('f1_score', ascending=False).head(15).to_string(index=False))
else:
    print("No finished runs found.")

if len(results_df) > 0:
    best_ssl_overall = results_df.sort_values('f1_score', ascending=False).iloc[0]
    print(f"\n{'='*100}")
    print("🏆 BEST SSL MODEL OVERALL")
    print(f"{'='*100}")
    print(f"Method: {best_ssl_overall['method']}")
    print(f"Unlabeled samples: {best_ssl_overall['n_unlabeled']:,.0f}")
    print(f"F1-Score: {best_ssl_overall['f1_score']:.4f}")
    print(f"ROC-AUC: {best_ssl_overall['roc_auc']:.4f}")
    print(f"Improvement: {best_ssl_overall['improvement_pct']:+.2f}%")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7d22fca57c10, raw_cell="import pandas as pd

ssl_sweep = api.sweep(f"{enti.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X30sZmlsZQ%3D%3D>,),kwargs {}:


AlreadyJoinedError: 


SSL RESULTS SUMMARY (Top 15 by F1-Score)
           method  n_unlabeled  f1_score  roc_auc  accuracy  precision   recall  improvement_pct           run_name   run_id
label_propagation        20000  0.690995 0.702938  0.616245   0.576285 0.862722         0.343987    smooth-sweep-29 qorygt1s
    self_training        15000  0.682927 0.752616  0.701589   0.724138 0.646154        -0.827683     peach-sweep-39 wmvxgd02
label_propagation        20000  0.682547 0.708477  0.615656   0.579208 0.830769        -0.882783     solar-sweep-22 7cit7ash
    self_training         5000  0.682249 0.755865  0.703943   0.731707 0.639053        -0.926131      lucky-sweep-7 m0on6hyr
label_propagation        20000  0.681886 0.708520  0.622719   0.587179 0.813018        -0.978850     quiet-sweep-27 js5ukvad
    self_training        15000  0.680905 0.750459  0.701001   0.725569 0.641420        -1.121356   restful-sweep-46 px55zn7z
      co_training        15000  0.680851 0.759887  0.699823   0.722444 0.643787    

AlreadyJoinedError: 

In [20]:
print(f"\n{'='*100}")
print("🥇 BEST MODEL PER SSL METHOD")
print(f"{'='*100}")

method_best_models = {}

for method in results_df['method'].unique():
    method_df = results_df[results_df['method'] == method].sort_values('f1_score', ascending=False)
    if len(method_df) > 0:
        method_best = method_df.iloc[0]
        method_best_models[method] = method_best
        
        print(f"\n{method.upper().replace('_', ' ')}:")
        print(f"  Unlabeled samples: {method_best['n_unlabeled']:,.0f}")
        print(f"  F1-Score: {method_best['f1_score']:.4f}")
        print(f"  ROC-AUC: {method_best['roc_auc']:.4f}")
        print(f"  Accuracy: {method_best['accuracy']:.4f}")
        print(f"  Improvement: {method_best['improvement_pct']:+.2f}%")
        print(f"  Run: {method_best['run_name']}")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7d2302959bd0, raw_cell="print(f"\n{'='*100}")
print("🥇 BEST MODEL PER SSL .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X31sZmlsZQ%3D%3D>,),kwargs {}:


AlreadyJoinedError: 


🥇 BEST MODEL PER SSL METHOD

SELF TRAINING:
  Unlabeled samples: 15,000
  F1-Score: 0.6829
  ROC-AUC: 0.7526
  Accuracy: 0.7016
  Improvement: -0.83%
  Run: peach-sweep-39

LABEL PROPAGATION:
  Unlabeled samples: 20,000
  F1-Score: 0.6910
  ROC-AUC: 0.7029
  Accuracy: 0.6162
  Improvement: +0.34%
  Run: smooth-sweep-29

CO TRAINING:
  Unlabeled samples: 15,000
  F1-Score: 0.6809
  ROC-AUC: 0.7599
  Accuracy: 0.6998
  Improvement: -1.13%
  Run: soft-sweep-28
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7d230294cc90, execution_count=20 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7d2302959bd0, raw_cell="print(f"\n{'='*100}")
print("🥇 BEST MODEL PER SSL .." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#

AlreadyJoinedError: 

In [21]:
if len(results_df) > 0:
    run = wandb.init(project=PROJECT, job_type="register-best-ssl-model")
    
    best_ssl = results_df.sort_values('f1_score', ascending=False).iloc[0]
    best_ssl_run_obj = api.run(f"{entity}/{PROJECT}/{best_ssl['run_id']}")
    
    print(f"\n{'='*100}")
    print(f"Registering BEST SSL Model: {best_ssl['method']}")
    print(f"{'='*100}")
    
    # Retrain with best config
    n_unlabeled = int(best_ssl['n_unlabeled'])
    X_unlabeled = X_unlabeled_full[:n_unlabeled]
    
    method = best_ssl['method']
    
    if method == 'label_propagation':
        X_combined = np.vstack([X_train.values, X_unlabeled])
        y_combined = np.concatenate([y_train.values, np.full(n_unlabeled, -1)])
        
        model = LabelPropagation(
            kernel='rbf',
            gamma=best_ssl_run_obj.config.get('lp_gamma', 0.1),
            max_iter=best_ssl_run_obj.config.get('lp_max_iter', 1000),
            n_jobs=-1
        )
        model.fit(X_combined, y_combined)
    
    elif method == 'label_spreading':
        X_combined = np.vstack([X_train.values, X_unlabeled])
        y_combined = np.concatenate([y_train.values, np.full(n_unlabeled, -1)])
        
        model = LabelSpreading(
            kernel='rbf',
            gamma=best_ssl_run_obj.config.get('lp_gamma', 0.1),
            alpha=best_ssl_run_obj.config.get('ls_alpha', 0.2),
            max_iter=best_ssl_run_obj.config.get('lp_max_iter', 1000),
            n_jobs=-1
        )
        model.fit(X_combined, y_combined)
    
    elif method == 'self_training':
        X_combined = np.vstack([X_train.values, X_unlabeled])
        y_combined = np.concatenate([y_train.values, np.full(n_unlabeled, -1)])
        
        base_clf = RandomForestClassifier(
            n_estimators=best_baseline_config['n_estimators'],
            max_depth=best_baseline_config['max_depth'],
            min_samples_split=best_baseline_config['min_samples_split'],
            min_samples_leaf=best_baseline_config['min_samples_leaf'],
            max_features=best_baseline_config['max_features'],
            random_state=42,
            n_jobs=-1,
            class_weight='balanced'
        )
        model = SelfTrainingClassifier(
            base_estimator=base_clf,
            threshold=best_ssl_run_obj.config.get('st_threshold', 0.75),
            max_iter=best_ssl_run_obj.config.get('st_max_iter', 10)
        )
        model.fit(X_combined, y_combined)
    
    elif method == 'co_training':
        # Save co-training as ensemble
        n_features = X_train.shape[1]
        mid = n_features // 2
        
        clf1 = RandomForestClassifier(**best_baseline_config, random_state=42, n_jobs=-1, class_weight='balanced')
        clf2 = RandomForestClassifier(**best_baseline_config, random_state=43, n_jobs=-1, class_weight='balanced')
        
        clf1.fit(X_train.values[:, :mid], y_train.values)
        clf2.fit(X_train.values[:, mid:], y_train.values)
        
        model = {'clf1': clf1, 'clf2': clf2, 'mid': mid, 'type': 'co_training'}
    
    # Save model
    model_filename = 'models/best_ssl_model.pkl'
    joblib.dump(model, model_filename)
    joblib.dump(scaler, 'models/scaler.pkl')
    
    # Create artifact
    artifact = wandb.Artifact(
        name='tox21-best-ssl-model',
        type='model',
        description=f'Best SSL model ({method}) for Tox21 toxicity prediction',
        metadata={
            'ssl_method': method,
            'n_unlabeled': n_unlabeled,
            'n_labeled': len(X_train),
            'f1_score': float(best_ssl['f1_score']),
            'roc_auc': float(best_ssl['roc_auc']),
            'accuracy': float(best_ssl['accuracy']),
            'improvement_pct': float(best_ssl['improvement_pct'])
        }
    )
    
    artifact.add_file(model_filename)
    artifact.add_file('models/scaler.pkl')
    run.log_artifact(artifact)
    run.finish()
    
    print(f"✓ Best SSL model ({method}) registered!")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7d230ab05b50, raw_cell="if len(results_df) > 0:
    run = wandb.init(proje.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X32sZmlsZQ%3D%3D>,),kwargs {}:


AlreadyJoinedError: 


Registering BEST SSL Model: label_propagation


✓ Best SSL model (label_propagation) registered!
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7d230abdae50, execution_count=21 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7d230ab05b50, raw_cell="if len(results_df) > 0:
    run = wandb.init(proje.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X32sZmlsZQ%3D%3D> result=None>,),kwargs {}:


AlreadyJoinedError: 

## Phase 5: Production Monitoring & Maintenance

Simulate a **production environment** where the model serves toxicity predictions in real-time. We track key performance indicators to detect model degradation.

### Monitored Metrics:
- **Prediction Latency**: Response time per request (target < 40ms)
- **Prediction Confidence**: Model certainty in predictions
- **Data Drift**: Distribution shift in input molecules
- **Endpoint Predictions**: Toxicity scores for nuclear receptor/stress response assays

### Monitoring Strategy:
- Log all metrics to W&B in real-time
- Set up alerts for threshold violations
- Track 100 simulated prediction requests
- Identify performance bottlenecks

This phase ensures the model remains healthy and performant in production.

In [22]:
import time
import random
from datetime import datetime

wandb.init(
    project=PROJECT,
    job_type="monitor-production",
    name=f"production-monitoring-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    tags=["production", "monitoring", "tox21"]
)

NUM_REQUESTS = 100
ALERT_THRESHOLD_MS = 40
CONFIDENCE_THRESHOLD = 0.5

for i in range(NUM_REQUESTS):
    prediction_time_ms = random.uniform(10, 50)
    prediction_confidence = random.uniform(0.7, 0.99)
    
    predictions = {
        "NR-AR": random.uniform(0, 1),
        "NR-ER": random.uniform(0, 1),
        "SR-ARE": random.uniform(0, 1),
        "SR-p53": random.uniform(0, 1),
    }
    
    avg_prediction = np.mean(list(predictions.values()))
    
    data_drift_score = random.uniform(0, 0.3)
    
    wandb.log({
        "prediction_time_ms": prediction_time_ms,
        "prediction_confidence": prediction_confidence,
        "avg_prediction_score": avg_prediction,
        
        "data_drift_score": data_drift_score,
        
        **{f"pred_{endpoint}": score for endpoint, score in predictions.items()},
        
        "request_id": i,
        "timestamp": time.time()
    })
    
    if prediction_time_ms > ALERT_THRESHOLD_MS:
        print(f"Request {i}: High latency detected ({prediction_time_ms:.2f}ms)")
    
    if prediction_confidence < CONFIDENCE_THRESHOLD:
        print(f"Request {i}: Low confidence ({prediction_confidence:.2f})")
    
    time.sleep(0.5)
    
wandb.finish()

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7d22fcad32d0, raw_cell="import time
import random
from datetime import dat.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X34sZmlsZQ%3D%3D>,),kwargs {}:


AlreadyJoinedError: 

Request 4: High latency detected (46.10ms)
Request 10: High latency detected (45.60ms)
Request 15: High latency detected (44.95ms)
Request 16: High latency detected (45.35ms)
Request 22: High latency detected (42.63ms)
Request 29: High latency detected (41.35ms)
Request 40: High latency detected (40.49ms)
Request 44: High latency detected (45.56ms)
Request 46: High latency detected (40.84ms)
Request 48: High latency detected (48.89ms)
Request 58: High latency detected (40.25ms)
Request 59: High latency detected (41.97ms)
Request 60: High latency detected (44.02ms)
Request 62: High latency detected (42.55ms)
Request 69: High latency detected (41.33ms)
Request 72: High latency detected (43.87ms)
Request 74: High latency detected (49.54ms)
Request 75: High latency detected (41.81ms)
Request 76: High latency detected (42.37ms)
Request 79: High latency detected (46.87ms)
Request 82: High latency detected (43.22ms)
Request 93: High latency detected (49.50ms)
Request 94: High latency detected

avg_prediction_score,▂▄█▅▇▅▇▇▃▃▁▅▇▆▆▇▂▆▄▇▃▃▃▄▄▇▅▆▃▄▃▃▃▆▅▄▅▄▃▅
data_drift_score,▅▇▃▆▂▆▄▅█▅▅▆▆▆█▃▁▄▆▆▆▁▇█▄▃▄▄▆▄▄▃▇▂▄███▆▃
pred_NR-AR,▁▆▃▇▄▇▂▇▁▂▁▃▇▅▇▃█▂▆▄▄▃▃▅▆▅▆▅▁▆▄▄▄▃█▄▆▂▄▅
pred_NR-ER,▁▄▇▆▅▂█▄▂▄▆▅▁▅▆▃▃▅▆▂█▃▂▄▅▅▁█▄▅▃▁▆▄▄▆▃▃▂▅
pred_SR-ARE,█▇▁▅▃▃▃▂▅▇▁▆▅▆▆▂▁▆▂▃▆▁▄▆▃▆▃▅▅▅▄▆█▆▆▂▄▂▅▆
pred_SR-p53,█▅▁█▃█▆▇▆▇█▂█▃▃▆▃▄▁▄▇▆▃▂█▆▆▁▇▁▄▃▄▁▂▄▄▄▃▃
prediction_confidence,▆▄▂▅▂▃▇▄▁▇▆▅▁█▆▅▄▄█▄▄▅▅▄▃▃▇▅▂▁▃▁▇▆▃▇▆▃▁▅
prediction_time_ms,▄▆▄▇▇▆▂▄▆▄▇▁▆▂▅▂▆▇▁▄▇█▃▃▅▄██▄▁▃█▆▂▄▂▁▆▅▂
request_id,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_prediction_score,0.61047


Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7d22ebab5590, execution_count=22 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7d22fcad32d0, raw_cell="import time
import random
from datetime import dat.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X34sZmlsZQ%3D%3D> result=None>,),kwargs {}:


AlreadyJoinedError: 

## Phase 6: Automated Retraining & Closing the Loop

Implement an **automated MLOps pipeline** that continuously monitors model health and triggers retraining when performance degrades.

### Core Functionality:
1. **Health Monitoring**: Query W&B API for recent production metrics
2. **Degradation Detection**: Check against confidence and drift thresholds
3. **Automatic Triggering**: Launch retraining when thresholds are exceeded
4. **Team Alerts**: Send notifications via W&B alerts (Slack/Email integration)
5. **Full Traceability**: Log all retraining events with context

### Why This Matters:
- **Self-Healing System**: No manual intervention required
- **Continuous Improvement**: Models adapt to new data patterns
- **Proactive vs Reactive**: Catch issues before they impact users
- **Production-Ready**: Can be deployed with cron, Airflow, Lambda, or GitHub Actions

This completes the MLOps loop, ensuring your Tox21 toxicity prediction model remains accurate and reliable over time.

In [23]:
import wandb
import numpy as np
from datetime import datetime, timedelta


print("="*80)
print("AUTOMATED RETRAINING PIPELINE")
print("="*80)

# --- Configuration ---
PERFORMANCE_THRESHOLD_F1 = 0.75        
CONFIDENCE_THRESHOLD = 0.70             
DRIFT_THRESHOLD = 0.25                  
MONITORING_LOOKBACK_DAYS = 7            
AUTO_RETRAIN_ENABLED = True             


print("\nConnecting to Weights & Biases API...")
api = wandb.Api()
entity = api.default_entity

print(f"Connected to entity: {entity}")
print(f"Project: {PROJECT}")

# --- Step 2: Fetch Latest Monitoring Data ---
print(f"\nFetching monitoring runs (last {MONITORING_LOOKBACK_DAYS} days)...")
try:
    # Get all project runs
    all_project_runs = api.runs(f"{entity}/{PROJECT}")
    
    # Filter for monitoring runs
    monitor_runs = [run for run in all_project_runs if run.job_type == "monitor-production"]
    
    if not monitor_runs:
        print("No monitoring runs found. Run the production monitoring script first.")
        monitor_runs = []
    else:
        print(f"✓ Found {len(monitor_runs)} monitoring runs")
        
except Exception as e:
    print(f"Error fetching runs from W&B API: {e}")
    monitor_runs = []

# --- Step 3: Analyze Performance Metrics ---
if monitor_runs:
    print("\nAnalyzing model performance...")
    
    # Get the latest monitoring run
    latest_monitor_run = max(monitor_runs, key=lambda r: r.created_at)
    
    print(f"   Latest run: {latest_monitor_run.name}")
    print(f"   Created at: {latest_monitor_run.created_at}")
    
    # Fetch metrics from the run history
    try:
        history = latest_monitor_run.history()
        
        if not history.empty:
            # Calculate aggregate metrics
            avg_confidence = history['prediction_confidence'].mean() if 'prediction_confidence' in history.columns else 0
            avg_drift = history['data_drift_score'].mean() if 'data_drift_score' in history.columns else 0
            avg_latency = history['prediction_time_ms'].mean() if 'prediction_time_ms' in history.columns else 0
            num_predictions = len(history)
            
            print(f"\nPerformance Metrics:")
            print(f"   Average Confidence: {avg_confidence:.4f}")
            print(f"   Average Drift Score: {avg_drift:.4f}")
            print(f"   Average Latency: {avg_latency:.2f}ms")
            print(f"   Total Predictions: {num_predictions}")
            
            # --- Step 4: Evaluate if Retraining is Needed ---
            print(f"\nEvaluating against thresholds...")
            print(f"   Confidence threshold: {CONFIDENCE_THRESHOLD}")
            print(f"   Drift threshold: {DRIFT_THRESHOLD}")
            
            needs_retraining = False
            retraining_reasons = []
            
            # Check confidence threshold
            if avg_confidence < CONFIDENCE_THRESHOLD:
                needs_retraining = True
                retraining_reasons.append(f"Low confidence: {avg_confidence:.4f} < {CONFIDENCE_THRESHOLD}")
                print(f"Low confidence detected")
            else:
                print(f"Confidence is healthy")
            
            # Check drift threshold
            if avg_drift > DRIFT_THRESHOLD:
                needs_retraining = True
                retraining_reasons.append(f"High drift: {avg_drift:.4f} > {DRIFT_THRESHOLD}")
                print(f"High data drift detected")
            else:
                print(f"Drift is within acceptable range")
            
            # --- Step 5: Trigger Retraining if Needed ---
            if needs_retraining and AUTO_RETRAIN_ENABLED:
                print(f"\nPERFORMANCE DEGRADATION DETECTED!")
                print(f"   Reasons:")
                for reason in retraining_reasons:
                    print(f"      • {reason}")
                
                print(f"\nTRIGGERING AUTOMATED RETRAINING...")
                
                # Create a retraining trigger run
                alert_run = wandb.init(
                    project=PROJECT,
                    job_type="automated-retraining-trigger",
                    name=f"retrain-trigger-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
                    tags=["retraining", "automated", "triggered"],
                    config={
                        'trigger_reason': ', '.join(retraining_reasons),
                        'avg_confidence': avg_confidence,
                        'avg_drift': avg_drift,
                        'avg_latency': avg_latency,
                        'threshold_confidence': CONFIDENCE_THRESHOLD,
                        'threshold_drift': DRIFT_THRESHOLD,
                        'monitoring_run': latest_monitor_run.name
                    }
                )
                
                # Log the trigger event
                wandb.log({
                    'retraining_triggered': 1,
                    'avg_confidence': avg_confidence,
                    'avg_drift': avg_drift,
                    'timestamp': datetime.now().timestamp()
                })
                
                # Send alert to team
                wandb.alert(
                    title="Automated Retraining Triggered",
                    text=f"""Model performance has degraded and requires retraining.
                    
**Performance Issues:**
{chr(10).join(['• ' + r for r in retraining_reasons])}

**Metrics:**
• Average Confidence: {avg_confidence:.4f}
• Average Drift: {avg_drift:.4f}
• Average Latency: {avg_latency:.2f}ms

**Action:** A new hyperparameter sweep should be launched to find a better model.

**Monitoring Run:** {latest_monitor_run.name}
**Trigger Time:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
""",
                    level=wandb.AlertLevel.WARN
                )
                
                print(f"\nRetraining trigger logged to W&B")
                print(f"   Run URL: {alert_run.url}")
                print(f"   Alert sent to team")
                
                # --- Option to Launch New Sweep Automatically ---
                print(f"\nNext Steps:")
                print(f"   1. A new hyperparameter sweep should be launched")
                print(f"   2. You can automate this by uncommenting the code below")
                
                # UNCOMMENT TO ENABLE AUTOMATIC SWEEP LAUNCH
                """
                print(f"\nLaunching new SSL sweep...")
                
                # Use the SSL sweep config from earlier
                new_sweep_id = wandb.sweep(ssl_sweep_config, project=PROJECT)
                print(f"   ✓ New sweep created: {new_sweep_id}")
                
                # Optionally run some agents automatically
                # wandb.agent(new_sweep_id, train_ssl, count=10)
                """
                
                alert_run.finish()
                
            elif needs_retraining and not AUTO_RETRAIN_ENABLED:
                print(f"\nRETRAINING RECOMMENDED BUT AUTO-RETRAIN IS DISABLED")
                print(f"   Reasons:")
                for reason in retraining_reasons:
                    print(f"      • {reason}")
                print(f"\n   Enable automatic retraining by setting AUTO_RETRAIN_ENABLED = True")
                
            else:
                print(f"\nMODEL PERFORMANCE IS HEALTHY")
                print(f"   No retraining needed at this time")
                
        else:
            print("No metrics found in monitoring run history")
            
    except Exception as e:
        print(f"Error analyzing metrics: {e}")
        
else:
    print("\nSkipping: No monitoring data available")
    print("   Run the production monitoring script first (Phase 5)")

print("Automated retraining pipeline completed")

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7d22f8ee6990, raw_cell="import wandb
import numpy as np
from datetime impo.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/lokmane/Desktop/mini_projet/notebooks/tox21/tox21_mlops_life_cycle.ipynb#X36sZmlsZQ%3D%3D>,),kwargs {}:


AlreadyJoinedError: 

AUTOMATED RETRAINING PIPELINE

Connecting to Weights & Biases API...
Connected to entity: l-benhammadi-esi
Project: QSAR_MLOPS_TOX21

Fetching monitoring runs (last 7 days)...
✓ Found 1 monitoring runs

Analyzing model performance...
   Latest run: production-monitoring-20260129-114353
   Created at: 2026-01-29T10:37:38Z

Performance Metrics:
   Average Confidence: 0.8467
   Average Drift Score: 0.1558
   Average Latency: 29.37ms
   Total Predictions: 100

Evaluating against thresholds...
   Confidence threshold: 0.7
   Drift threshold: 0.25
Confidence is healthy
Drift is within acceptable range

MODEL PERFORMANCE IS HEALTHY
   No retraining needed at this time
Automated retraining pipeline completed
Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7d22ec3bc0d0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7d22ff6f72d0, execution_count=23 error_before_exec=None error_in_exec=None info=<ExecutionInfo 

AlreadyJoinedError: 